1、实现Adaboost方法对2个数据集进行分类分析，进行分类训练和测试，并给出分析结果；

2、采用已掌握算法（决策树、朴素贝叶斯、KNN或SVM）对数据集进行分析，并和Adaboost算法给出实验结果比较；

3、说明sklearn中Adaboost的算法原理（选做，读源码，写代码注释和算法流程）

实验要求：实验报告中要包含存储结构说明，算法思路设计，运行时的输入输出截图，算法源码中添加详细注释，写明算法流程等

注意：作业提交需包含所有代码，必要时可提交压缩包，代码尽可能在(姓名_集成学习.ipynb)中

In [42]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score


In [43]:
wine=pd.read_csv('./dataset/winequality-red.csv',";")
data_wine=wine.iloc[:,:-1]
data_wine

C:\Users\X2931\AppData\Local\Temp\ipykernel_18512\1776900977.py:1: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  wine=pd.read_csv('./dataset/winequality-red.csv',";")


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2


In [44]:
target_wine=wine.iloc[:,-1]
target_wine= np.where(target_wine>=6, int(1), int(0))
target_wine

array([0, 0, 0, ..., 1, 0, 1])

In [62]:
X_tarin,X_test,y_train,y_test=train_test_split(data_wine.values,target_wine,test_size=0.75,random_state=42)

基分类器采取决策树
在fit方法中，我们首先初始化权重向量w，将每个样本的权重设置为1/n_samples。然后，我们迭代n_estimators次，对于每次迭代，我们使用DecisionTreeClassifier作为弱分类器，并使用当前权重向量来拟合模型。然后，我们用训练数据预测标签，并计算当前弱分类器的错误率。接下来，我们计算弱分类器的权重alpha，并更新权重向量w。最后，我们保存弱分类器和相应的权重alpha。

在predict方法中，我们首先初始化预测标签向量y_pred为全零向量，并对于每个弱分类器，使用其预测函数对输入数据进行预测，并根据其权重alpha对弱分类器的预测结果进行加权。最后，我们返回一个二元标签向量，其中正数表示正类，负数表示负类。

In [63]:
class Adaboost:
    def __init__(self, n_classers):
        self.n_estimators = n_classers
        self.estimators = []
        self.alphas = []
        
    def fit(self, X, y):
        n_samples, _ = X.shape
        w = np.full(n_samples, (1 / n_samples))
        
        for i in range(self.n_estimators):

            clf = DecisionTreeClassifier(max_depth=8, random_state=0)
            clf.fit(X, y, sample_weight=w)
            y_pred = clf.predict(X)

            error = np.sum(w * (y != y_pred))
            alpha = 0.5 * np.log((1 - error) / error)
            self.alphas.append(alpha)

            w *= np.exp(-alpha * y * y_pred)
            w /= np.sum(w)

            self.estimators.append(clf)
            
    def predict(self, X):
        y_pred = np.zeros(X.shape[0])
        
        for i in range(self.n_estimators):
            y_pred += self.alphas[i] * self.estimators[i].predict(X)
            
        return np.sign(y_pred).astype(int)


In [64]:
# 在wine数据集上测试Adaboost
Ada = Adaboost(n_classers=1000)
Ada.fit(X_tarin,y_train)
y_pred = Ada.predict(X_test)
acc_wine = accuracy_score(y_test, y_pred)
print("Accuracy on wine dataset using Adaboost: {:.2f}%".format(acc_wine * 100))

Accuracy on wine dataset using Adaboost: 59.00%
